In [ ]:
%matplotlib inline
import pandas as pd
from rich import print
import numpy as np
import matplotlib.pyplot as plt
#from quickbin import bin2d
from scipy.stats import binned_statistic_2d
import pdr
from astropy.visualization import ZScaleInterval
from astropy import wcs as pywcs
import warnings
from pyarrow import ArrowInvalid

In [ ]:
catfiles = !ls data/*/*nd*catalog*

In [ ]:
tbl = pd.DataFrame()
for f in catfiles:
    # if f in ['data/e03915/e03915-nd-10-catalog.parquet',
    #          'data/e03915/e03915-nd-11-catalog.parquet']:
        # continue
    try:
        tbl = pd.concat([tbl,pd.read_parquet(f)])
    except ArrowInvalid:
        print(f'Unable to open {f}')
        continue

In [ ]:
tbl.to_parquet('/Users/cm/Downloads/foo.parquet')

In [ ]:
# NOTE: 'data/e03915/e03915-nd-10-catalog.parquet' is a misformed file somehow
# also 'data/e03915/e03915-nd-11-catalog.parquet'
# foo = pdr.read('data/e03915/e03915-nd-10-catalog.parquet')
# foo.load('all')
# foo['LABEL']
# import pyarrow.parquet as pq
# table = pq.read_table('data/e03915/e03915-nd-10-catalog.parquet',schema=None)
# import pandas as pd
# df = pd.read_parquet('data/e03915/e03915-nd-10-catalog.parquet')

In [ ]:
tbl

In [ ]:
plt.hist(tbl['NUV_MDL_SIGMA'],range=[0,10],bins=100);
plt.xlabel('SIGMA (arcsec)')
plt.title(f'n={len(np.where(np.isfinite(tbl['NUV_MDL_SIGMA']))[0])}')

In [ ]:
ix = np.where(
    (np.isfinite(tbl['NUV_MDL_SIGMA'])) & 
    (tbl['NUV_MDL_SIGMA']<10) & (tbl['NUV_MDL_SIGMA']>0.2) &
    (tbl['NUV_MDL_CPS']>1e-03) &
    (tbl['NUV_MDL_SIGMA']) & (tbl['NUV_MDL_SIGMA']) &
    (tbl['NUV_SOFTEDGE_FLAG_A6']==0) & (tbl['NUV_HARDEDGE_FLAG_A6']==0) &
    (tbl['NUV_GHOST_FLAG_A6']==0) & (tbl['NUV_HOTSPOT_FLAG_A6']==0))
plt.hist(tbl.iloc[ix]['NUV_MDL_SIGMA'],range=[0,10],bins=100);
plt.xlabel('SIGMA (arcsec)')
plt.title(f'n={len(np.where(np.isfinite(tbl.iloc[ix]['NUV_MDL_SIGMA']))[0])}')

In [ ]:
def counts2mag(cps, band):
    scale = 18.82 if band == 'FUV' else 20.08
    with np.errstate(invalid='ignore'):
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            mag = -2.5 * np.log10(cps) + scale
    return mag

In [ ]:
ix = np.where(
    (np.isfinite(tbl['NUV_MDL_SIGMA'])) & 
    (tbl['NUV_MDL_SIGMA']<10) & (tbl['NUV_MDL_SIGMA']>0.2) &
    #(tbl['NUV_MDL_CPS']>1e-03) &
    (tbl['NUV_MDL_SIGMA']) & (tbl['NUV_MDL_SIGMA']) &
    (tbl['NUV_SOFTEDGE_FLAG_A6']==0) & (tbl['NUV_HARDEDGE_FLAG_A6']==0) &
    (tbl['NUV_GHOST_FLAG_A6']==0) & (tbl['NUV_HOTSPOT_FLAG_A6']==0) &
    (counts2mag(tbl['NUV_MDL_CPS'].values,'NUV')<25))
plt.hist(tbl.iloc[ix]['NUV_MDL_SIGMA'],range=[0,10],bins=100);
plt.xlabel('SIGMA (arcsec)')
plt.title(f'n={len(np.where(np.isfinite(tbl.iloc[ix]['NUV_MDL_SIGMA']))[0])}')

In [ ]:
def imgpath_from_obsdata(eclipse,leg,rootpath='data'):
    estr = f'e{str(eclipse).zfill(5)}'
    return f'{rootpath}/{estr}/{estr}-nd-ffull-b{str(leg).zfill(2)}-image-r.fits'

In [ ]:
def make_wcs(hdr):
    #hdr = img['CNT_HEADER']
    wcs = pywcs.WCS(naxis=hdr['NAXIS'])
    wcs.wcs.cdelt = [hdr['CDELT1'],
                     hdr['CDELT2']]
    wcs.wcs.ctype = [hdr['CTYPE1'],
                     hdr['CTYPE2']]
    wcs.wcs.crpix = [hdr['CRPIX1'],
                     hdr['CRPIX2']]
    wcs.wcs.crval = [hdr['CRVAL1'],
                     hdr['CRVAL2']]
    return wcs

In [ ]:
# What are the stars that have a decent count rate but bonkers fwhm?
ix = np.where(
    (np.isfinite(tbl['NUV_MDL_SIGMA'])) & 
    #(tbl['NUV_MDL_SIGMA']<10) &
    (tbl['NUV_MDL_SIGMA']==10) &
    #(tbl['NUV_MDL_CPS']>1e-03) &
    (tbl['NUV_MDL_SIGMA']) & (tbl['NUV_MDL_SIGMA']) &
    (tbl['NUV_SOFTEDGE_FLAG_A6']==0) & (tbl['NUV_HARDEDGE_FLAG_A6']==0) &
    (tbl['NUV_GHOST_FLAG_A6']==0) & (tbl['NUV_HOTSPOT_FLAG_A6']==0) &
    (counts2mag(tbl['NUV_MDL_CPS'].values,'NUV')<25))

for cix in np.random.choice(ix[0],size=20):
    eclipse,leg = tbl.iloc[cix][['ECLIPSE','LEG']]
    img = pdr.read(
        imgpath_from_obsdata(eclipse,leg))
    img.load('CNT')
    wcs = make_wcs(img['CNT_HEADER'])
    # Note the following line is only valid _for the current observation_ and produces a lot of nonsense for other observations in the table
    catpos = wcs.wcs_world2pix(
        list(zip(
            tbl[(tbl['LEG']==leg) & (tbl['ECLIPSE']==eclipse)]['RA'].values,
            tbl[(tbl['LEG']==leg) & (tbl['ECLIPSE']==eclipse)]['DEC'].values)),
        1) # set the origin to FITS standard
    ra,dec = tbl.iloc[cix][['RA','DEC']]
    imgpos = wcs.wcs_world2pix([[ra,dec]],1) # set the origin to FITS standard
    imgx,imgy = imgpos[0]
    imsz = np.shape(img['CNT'])[-2:]
    
    # crop on the subframe
    # noting that image coordinates and numpy coordinates are flipped
    boxsz = 50
    x1, x2, y1, y2 = (max(int(imgy - boxsz), 0),
                      min(int(imgy + boxsz), imsz[0]),
                      max(int(imgx - boxsz), 0),
                      min(int(imgx + boxsz), imsz[1]))
    #plt.plot([50],[50],'rx')
    plt.figure(figsize=(5,5))
    plt.title(cix)
    
    plt.imshow(img['CNT'],
               origin="lower",
               cmap="Greys_r",
               vmin=np.mean(img['CNT'][x1:x2, y1:y2]),
               vmax=np.max(img['CNT'][x1:x2, y1:y2]))
    plt.xlim([y1,y2])
    plt.ylim([x1,x2])
    plt.plot(catpos[:,0],catpos[:,1],'yx')

In [ ]:
tbl[(tbl['LEG']==0) & (tbl['ECLIPSE']==3909)]

In [ ]:
# random sample of all sources presumed to be reasonable
ix = np.where(
    (np.isfinite(tbl['NUV_MDL_SIGMA'])) & 
    (tbl['NUV_MDL_SIGMA']<10) &
    (tbl['NUV_MDL_SIGMA']>0.2) &
    #(tbl['NUV_MDL_CPS']>1e-03) &
    (tbl['NUV_MDL_SIGMA']) & (tbl['NUV_MDL_SIGMA']) &
    (tbl['NUV_SOFTEDGE_FLAG_A6']==0) & (tbl['NUV_HARDEDGE_FLAG_A6']==0) &
    (tbl['NUV_GHOST_FLAG_A6']==0) & (tbl['NUV_HOTSPOT_FLAG_A6']==0) &
    (counts2mag(tbl['NUV_MDL_CPS'].values,'NUV')<25))

for cix in np.random.choice(ix[0],size=20):
    eclipse,leg = tbl.iloc[cix][['ECLIPSE','LEG']]
    img = pdr.read(
        imgpath_from_obsdata(eclipse,leg))
    img.load('CNT')
    wcs = make_wcs(img['CNT_HEADER'])
    # Note the following line is only valid _for the current observation_ and produces a lot of nonsense for other observations in the table
    catpos = wcs.wcs_world2pix(
        list(zip(
            tbl[(tbl['LEG']==leg) & (tbl['ECLIPSE']==eclipse)]['RA'].values,
            tbl[(tbl['LEG']==leg) & (tbl['ECLIPSE']==eclipse)]['DEC'].values)),
        1) # set the origin to FITS standard
    ra,dec = tbl.iloc[cix][['RA','DEC']]
    imgpos = wcs.wcs_world2pix([[ra,dec]],1) # set the origin to FITS standard
    imgx,imgy = imgpos[0]
    imsz = np.shape(img['CNT'])[-2:]
    
    # crop on the subframe
    # noting that image coordinates and numpy coordinates are flipped
    boxsz = 50
    x1, x2, y1, y2 = (max(int(imgy - boxsz), 0),
                      min(int(imgy + boxsz), imsz[0]),
                      max(int(imgx - boxsz), 0),
                      min(int(imgx + boxsz), imsz[1]))
    #plt.plot([50],[50],'rx')
    plt.figure(figsize=(5,5))
    plt.title(f'{eclipse} {leg}')
    
    plt.imshow(img['CNT'],
               origin="lower",
               cmap="Greys_r",
               vmin=np.mean(img['CNT'][x1:x2, y1:y2]),
               vmax=np.max(img['CNT'][x1:x2, y1:y2]))
    plt.xlim([y1,y2])
    plt.ylim([x1,x2])
    plt.plot(catpos[:,0],catpos[:,1],'yx')

In [ ]:
aper_photom = {'cps':tbl['NUV_CPS_A4'],
               'bg':(9**2)*(tbl['NUV_CPS_A6']-tbl['NUV_CPS_A5'])/(17.3**2-12.8**2),
              }
modl_photom = {'cps':tbl['NUV_MDL_CPS'],
               'bg':tbl['NUV_MDL_BKG_CPS']
              }

plt.figure(figsize=(10,10))
plt.plot(aper_photom['cps']-aper_photom['bg'],
         modl_photom['cps'],'k.',alpha=0.4)
plt.xlim([0,500]);plt.ylim([0,500]);
plt.plot([0,500],[0,500],'k:',alpha=0.5)
plt.xlabel('aperture photometry (aper4 cps-bg)')
plt.ylabel('model photometry (cps)')

plt.figure(figsize=(10,10))
plt.plot(aper_photom['cps']-aper_photom['bg'],
         modl_photom['cps'],'k.',alpha=0.4)
plt.xlim([0,50]);plt.ylim([0,50]);
plt.plot([0,500],[0,500],'k:',alpha=0.5)
plt.xlabel('aperture photometry')
plt.ylabel('model photometry')

plt.figure(figsize=(10,10))
plt.plot(aper_photom['cps']-aper_photom['bg'],
         modl_photom['cps'],'k.',alpha=0.4)
plt.xlim([0,10]);plt.ylim([0,10]);
plt.plot([0,50],[0,50],'k:',alpha=0.3)
plt.xlabel('aperture photometry')
plt.ylabel('model photometry')

sigmas = tbl['NUV_MDL_SIGMA']
ix = np.where((np.isfinite(sigmas)) & 
              (sigmas<6) & (sigmas>0.5) &
              (tbl['NUV_SOFTEDGE_FLAG_A6']==0) & (tbl['NUV_HARDEDGE_FLAG_A6']==0) &
              (tbl['NUV_GHOST_FLAG_A6']==0) & (tbl['NUV_HOTSPOT_FLAG_A6']==0))
print(f'{100*np.round(len(ix[0])/len(sigmas),3)}% sources retained')

#plt.figure(figsize=(10,10))
plt.plot((aper_photom['cps']-aper_photom['bg']).iloc[ix],
         modl_photom['cps'].iloc[ix],
         'b.',alpha=0.1)
plt.xlim([0,10]);plt.ylim([0,10]);
plt.plot([0,50],[0,50],'k:',alpha=1)
plt.xlabel('aperture photometry (aper4 cps-bg)')
plt.ylabel('model photometry (cps)')

print(len(ix[0]))
print(len(sigmas))

In [ ]:
aper_photom = {'cps':tbl['NUV_CPS_A4'],
               'bg':(9**2)*(tbl['NUV_CPS_A6']-tbl['NUV_CPS_A5'])/(17.3**2-12.8**2),
              }
modl_photom = {'cps':tbl['NUV_MDL_CPS'],
               'bg':tbl['NUV_MDL_BKG_CPS']
              }

# Create figure with three subplots side by side
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(30, 10))

# Panel 1: Full range (0-500)
ax1.plot(aper_photom['cps']-aper_photom['bg'],
         modl_photom['cps'],'k.',alpha=0.4)
ax1.set_xlim([0,500])
ax1.set_ylim([0,500])
ax1.plot([0,500],[0,500],'k:',alpha=0.5)
ax1.set_xlabel('aperture photometry (aper4 cps-bg)')
ax1.set_ylabel('model photometry (cps)')

# Panel 2: Medium range (0-50)
ax2.plot(aper_photom['cps']-aper_photom['bg'],
         modl_photom['cps'],'k.',alpha=0.4)
ax2.set_xlim([0,50])
ax2.set_ylim([0,50])
ax2.plot([0,500],[0,500],'k:',alpha=0.5)
ax2.set_xlabel('aperture photometry')
ax2.set_ylabel('model photometry')

# Panel 3: Close-up range (0-10) with filtered data
ax3.plot(aper_photom['cps']-aper_photom['bg'],
         modl_photom['cps'],'k.',alpha=0.4)

# Apply filtering
sigmas = tbl['NUV_MDL_SIGMA']
ix = np.where((np.isfinite(sigmas)) & 
              (sigmas<6) & (sigmas>0.5) &
              (tbl['NUV_SOFTEDGE_FLAG_A6']==0) & (tbl['NUV_HARDEDGE_FLAG_A6']==0) &
              (tbl['NUV_GHOST_FLAG_A6']==0) & (tbl['NUV_HOTSPOT_FLAG_A6']==0))

print(f'{100*np.round(len(ix[0])/len(sigmas),3)}% sources retained')
print(f'{len(ix[0])} sources out of {len(sigmas)} total')

# Add filtered data points
ax3.plot((aper_photom['cps']-aper_photom['bg']).iloc[ix],
         modl_photom['cps'].iloc[ix],
         'b.',alpha=0.1)
ax3.set_xlim([0,10])
ax3.set_ylim([0,10])
ax3.plot([0,50],[0,50],'k:',alpha=1)
ax3.set_xlabel('aperture photometry (aper4 cps-bg)')
ax3.set_ylabel('model photometry (cps)')

plt.tight_layout()
plt.show()

In [ ]:
aper_photom = {'cps':tbl['NUV_CPS_A4'],
               'bg':(9**2)*(tbl['NUV_CPS_A6']-tbl['NUV_CPS_A5'])/(17.3**2-12.8**2),
              }
modl_photom = {'cps':tbl['NUV_MDL_CPS'],
               'bg':tbl['NUV_MDL_BKG_CPS']
              }

# Create figure with three subplots side by side
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(30, 10))

# Panel 1: Full range (0-500)
ax1.plot(aper_photom['cps']-aper_photom['bg'],
         modl_photom['cps'],'k.',alpha=0.4)
ax1.set_xlim([0,500])
ax1.set_ylim([0,500])
ax1.plot([0,500],[0,500],'k:',alpha=0.5)
ax1.set_xlabel('aperture photometry (aper4 cps-bg)')
ax1.set_ylabel('model photometry (cps)')
ax1.set_title('Full View (0-500)', fontsize=14, fontweight='bold')

# Add zoom boxes to show what's being zoomed into
from matplotlib.patches import Rectangle
zoom_box1 = Rectangle((0, 0), 50, 50, linewidth=2, edgecolor='red', facecolor='none', alpha=0.8)
ax1.add_patch(zoom_box1)
zoom_box2 = Rectangle((0, 0), 10, 10, linewidth=2, edgecolor='blue', facecolor='none', alpha=0.8)
ax1.add_patch(zoom_box2)

# Panel 2: Medium range (0-50)
ax2.plot(aper_photom['cps']-aper_photom['bg'],
         modl_photom['cps'],'k.',alpha=0.4)
ax2.set_xlim([0,50])
ax2.set_ylim([0,50])
ax2.plot([0,500],[0,500],'k:',alpha=0.5)
ax2.set_xlabel('aperture photometry')
ax2.set_ylabel('model photometry')
ax2.set_title('Medium Zoom (0-50)', fontsize=14, fontweight='bold', color='red')

# Add zoom box for final zoom level
zoom_box3 = Rectangle((0, 0), 10, 10, linewidth=2, edgecolor='blue', facecolor='none', alpha=0.8)
ax2.add_patch(zoom_box3)

# Add border to indicate this is a zoom of panel 1
for spine in ax2.spines.values():
    spine.set_edgecolor('red')
    spine.set_linewidth(3)

# Panel 3: Close-up range (0-10) with filtered data
ax3.plot(aper_photom['cps']-aper_photom['bg'],
         modl_photom['cps'],'k.',alpha=0.4)

# Apply filtering
sigmas = tbl['NUV_MDL_SIGMA']
ix = np.where((np.isfinite(sigmas)) & 
              (sigmas<6) & (sigmas>0.5) &
              (tbl['NUV_SOFTEDGE_FLAG_A6']==0) & (tbl['NUV_HARDEDGE_FLAG_A6']==0) &
              (tbl['NUV_GHOST_FLAG_A6']==0) & (tbl['NUV_HOTSPOT_FLAG_A6']==0))

print(f'{100*np.round(len(ix[0])/len(sigmas),3)}% sources retained')
print(f'{len(ix[0])} sources out of {len(sigmas)} total')

# Add filtered data points
ax3.plot((aper_photom['cps']-aper_photom['bg']).iloc[ix],
         modl_photom['cps'].iloc[ix],
         'b.',alpha=0.1)
ax3.set_xlim([0,10])
ax3.set_ylim([0,10])
ax3.plot([0,50],[0,50],'k:',alpha=1)
ax3.set_xlabel('aperture photometry (aper4 cps-bg)')
ax3.set_ylabel('model photometry (cps)')
ax3.set_title('Maximum Zoom (0-10)', fontsize=14, fontweight='bold', color='blue')

# Add border to indicate this is a zoom of panel 2
for spine in ax3.spines.values():
    spine.set_edgecolor('blue')
    spine.set_linewidth(3)

# Add zoom arrows between panels
fig.text(0.31, 0.5, '→', fontsize=40, color='red', ha='center', va='center', weight='bold')
fig.text(0.64, 0.5, '→', fontsize=40, color='blue', ha='center', va='center', weight='bold')

# Add overall figure title
fig.suptitle('Aperture vs Model Photometry: Progressive Zoom Levels', fontsize=16, fontweight='bold', y=0.95)

plt.tight_layout()
plt.subplots_adjust(top=0.88)  # Make room for the suptitle
plt.show()

In [ ]:
# Create figure with two subplots side by side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

# Panel 1: Limited Y range (0-50)
ax1.plot(tbl['NUV_MDL_SIGMA'],tbl['NUV_MDL_CPS'],'k.',alpha=0.4)
ax1.set_xlim([0,10])
ax1.set_ylim([0,50])
ax1.set_xlabel('SIGMA')
ax1.set_ylabel('CPS')
ax1.set_title('Limited Y View (0-50 CPS)', fontsize=14, fontweight='bold')

# Add zoom indication showing the full range being zoomed out
from matplotlib.patches import Rectangle
# Get the actual data range to show what's being cut off
y_max = tbl['NUV_MDL_CPS'].max()
zoom_box = Rectangle((0, 50), 10, y_max-50, linewidth=2, edgecolor='red', 
                    facecolor='red', alpha=0.1, linestyle='--')
ax1.add_patch(zoom_box)
ax1.text(5, 25, f'Data extends to\n~{int(y_max)} CPS', ha='center', va='center', 
         bbox=dict(boxstyle="round,pad=0.3", facecolor='yellow', alpha=0.7))

# Panel 2: Full Y range (auto-scaled)
ax2.plot(tbl['NUV_MDL_SIGMA'],tbl['NUV_MDL_CPS'],'k.',alpha=0.4)
ax2.set_xlim([0,10])
# Let matplotlib auto-scale the y-axis to show full range
ax2.set_xlabel('SIGMA')
ax2.set_ylabel('CPS')
ax2.set_title('Full Y Range (Auto-scaled)', fontsize=14, fontweight='bold', color='red')

# Add border to indicate this is the full view
for spine in ax2.spines.values():
    spine.set_edgecolor('red')
    spine.set_linewidth(3)

# Add a horizontal line at y=50 to show the cutoff from panel 1
ax2.axhline(y=50, color='red', linestyle='--', alpha=0.7, linewidth=2)
ax2.text(8, 50, 'Panel 1 cutoff', ha='right', va='bottom', color='red', fontweight='bold')

# Add zoom arrow between panels
fig.text(0.5, 0.5, '→', fontsize=40, color='red', ha='center', va='center', weight='bold')

# Add overall figure title
fig.suptitle('NUV Model Sigma vs CPS: Y-axis Range Comparison', fontsize=16, fontweight='bold', y=0.95)

plt.tight_layout()
plt.subplots_adjust(top=0.88)  # Make room for the suptitle
plt.show()

In [ ]:
# Create figure with four subplots: 2x2 grid
fig = plt.figure(figsize=(20, 12))

# Create gridspec for custom layout
gs = fig.add_gridspec(2, 2, height_ratios=[3, 1], hspace=0.05)

# Main scatter plots
ax1 = fig.add_subplot(gs[0, 0])  # Top left
ax2 = fig.add_subplot(gs[0, 1])  # Top right

# Histogram plots (inverted)
ax1_hist = fig.add_subplot(gs[1, 0], sharex=ax1)  # Bottom left
ax2_hist = fig.add_subplot(gs[1, 1], sharex=ax2)  # Bottom right

# Panel 1: Limited Y range (0-50)
ax1.plot(tbl['NUV_MDL_SIGMA'],tbl['NUV_MDL_CPS'],'k.',alpha=0.4)
ax1.set_xlim([0,10])
ax1.set_ylim([0,50])
ax1.set_xlabel('SIGMA')
ax1.set_ylabel('CPS')
ax1.set_title('Limited Y View (0-50 CPS)', fontsize=14, fontweight='bold')

# Add zoom indication showing the full range being zoomed out
from matplotlib.patches import Rectangle
# Get the actual data range to show what's being cut off
y_max = tbl['NUV_MDL_CPS'].max()
zoom_box = Rectangle((0, 50), 10, y_max-50, linewidth=2, edgecolor='red', 
                    facecolor='red', alpha=0.1, linestyle='--')
ax1.add_patch(zoom_box)
ax1.text(5, 25, f'Data extends to\n~{int(y_max)} CPS', ha='center', va='center', 
         bbox=dict(boxstyle="round,pad=0.3", facecolor='yellow', alpha=0.7))

# Panel 2: Full Y range (auto-scaled)
ax2.plot(tbl['NUV_MDL_SIGMA'],tbl['NUV_MDL_CPS'],'k.',alpha=0.4)
ax2.set_xlim([0,10])
# Let matplotlib auto-scale the y-axis to show full range
ax2.set_xlabel('SIGMA')
ax2.set_ylabel('CPS')
ax2.set_title('Full Y Range (Auto-scaled)', fontsize=14, fontweight='bold', color='red')

# Add border to indicate this is the full view
for spine in ax2.spines.values():
    spine.set_edgecolor('red')
    spine.set_linewidth(3)

# Add a horizontal line at y=50 to show the cutoff from panel 1
ax2.axhline(y=50, color='red', linestyle='--', alpha=0.7, linewidth=2)
ax2.text(8, 50, 'Panel 1 cutoff', ha='right', va='bottom', color='red', fontweight='bold')

# Add zoom arrow between panels
fig.text(0.5, 0.65, '→', fontsize=40, color='red', ha='center', va='center', weight='bold')

# Create histograms along SIGMA axis (inverted)
sigma_data = tbl['NUV_MDL_SIGMA']
hist_bins = 100

# Filter out any NaN or infinite values
sigma_clean = sigma_data[np.isfinite(sigma_data)]

# Histogram for panel 1 - create upside down bars
n1, bins1, patches1 = ax1_hist.hist(sigma_clean, bins=hist_bins, range=(0, 10), 
                                    color='gray', alpha=0.7, bottom=0)
# Manually invert the bars by setting their y-coordinates to negative
for patch in patches1:
    patch.set_y(-patch.get_height())
    
# Set the y-axis limits to show negative values (inverted histogram)
max_count1 = max(n1) if len(n1) > 0 else 1000
ax1_hist.set_ylim(-max_count1 * 1.1, 0)
# ax1_hist.set_ylabel('Count')
ax1_hist.set_xlabel('SIGMA')
ax1_hist.grid(True, alpha=0.3)
ax1_hist.set_yticks([])

# Histogram for panel 2 - create upside down bars  
n2, bins2, patches2 = ax2_hist.hist(sigma_clean, bins=hist_bins, range=(0, 10), 
                                    color='gray', alpha=0.7, bottom=0)
# Manually invert the bars by setting their y-coordinates to negative
for patch in patches2:
    patch.set_y(-patch.get_height())
    
# Set the y-axis limits to show negative values (inverted histogram)
max_count2 = max(n2) if len(n2) > 0 else 1000
ax2_hist.set_ylim(-max_count2 * 1.1, 0)
# ax2_hist.set_ylabel('Count')
ax2_hist.set_xlabel('SIGMA')
ax2_hist.grid(True, alpha=0.3)
ax2_hist.set_yticks([])

# Remove x-axis labels from top plots since histograms show them
ax1.set_xlabel('')
ax2.set_xlabel('')
ax1.tick_params(labelbottom=False)
ax2.tick_params(labelbottom=False)

# Add overall figure title
fig.suptitle('NUV Model Sigma vs CPS: Y-axis Range Comparison with SIGMA Distribution', fontsize=16, fontweight='bold', y=0.95)

plt.subplots_adjust(top=0.88, hspace=0.1)  # Make room for the suptitle and reasonable gap
plt.show()

In [ ]:
ix = np.where(
    (np.isfinite(tbl['NUV_MDL_SIGMA'])) & 
    (tbl['NUV_MDL_SIGMA']<10) &
    (tbl['NUV_MDL_SIGMA']>0.2) &
    #(tbl['NUV_MDL_CPS']>1e-03) &
    (tbl['NUV_MDL_SIGMA']) & (tbl['NUV_MDL_SIGMA']) &
    (tbl['NUV_SOFTEDGE_FLAG_A6']==0) & (tbl['NUV_HARDEDGE_FLAG_A6']==0) &
    (tbl['NUV_GHOST_FLAG_A6']==0) & (tbl['NUV_HOTSPOT_FLAG_A6']==0) &
    (counts2mag(tbl['NUV_MDL_CPS'].values,'NUV')<25))
plt.plot(tbl.iloc[ix]['RA'],tbl.iloc[ix]['DEC'],'.k',alpha=0.1)

In [ ]:
# plt.figure(figsize=(12,12))
# #plt.plot(tbl['NUV_MAG_A4'],tbl['FUV_MAG_A4'],'.k',alpha=0.1)
# plt.plot(tbl['NUV_MAG_A4'].iloc[ix],
#          tbl['FUV_MAG_A4'].iloc[ix],'.b',alpha=0.1)

# plt.xlabel('NUV MAG A4')
# plt.ylabel('FUV MAG A4')

plt.figure(figsize=(12,5))
# Use a logarithmic stretch for the color scale to enhance visibility of both dense and sparse regions
h = plt.hist2d(
    tbl['NUV_MAG_A4'].iloc[ix],
    tbl['FUV_MAG_A4'].iloc[ix],
    bins=100,
    range=[[15,22],[15,25]],
    cmap='Greys',
    norm=plt.matplotlib.colors.LogNorm(vmin=0.1)
)
plt.colorbar(h[3], label='Counts (log scale)')
plt.xlabel('NUV MAG A4')
plt.ylabel('FUV MAG A4')
plt.title('NUV vs FUV Magnitude (log-stretched 2D histogram)')

In [ ]:
for k in tbl.keys():
    print(k)